### L5: Data Analyst Agent 🤓

First start by filtering warnings and loading environment variables.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from helper import load_env
load_env()

Now, let's create the data analyst agent. You will create a sandbox, then open the `pokemon.csv` dataset and write it to the sandbox as `data.csv`.

In [4]:
from lib.utils import create_sandbox

sbx = create_sandbox()

with open("pokemon.csv", "r", encoding="utf-8") as f:
    content = f.read()

sbx.files.write("data.csv", content)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=ieu55eqyr43qqr4umb7b3)

WriteInfo(name='data.csv', type='file', path='/home/user/data.csv')

Now you can import your `coding_agent` and modify the system prompt to let it know that the user has uploaded a dataset called `data.csv` and it should create interesting plots.

In [5]:
from lib.coding_agent import coding_agent, log
from lib.tools_schemas import execute_code_schema
from lib.tools import execute_code
from openai import OpenAI

client = OpenAI()

system = """You are a senior python programmer. 
You must run the code using the `execute_code` tool.
The user has uploaded a data.csv.
You help the user understanding the data 
by creating interesting plots.
"""

tools = { "execute_code" : execute_code }

Now you can query the data as normal and the sandbox will execute code on the data to answer the queries.

In [6]:
messages = []

query = "What is the data about?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"import pandas as pd\n\ndf = pd.read_csv('data.csv')\ndf.info(), df.head()"}                 │
│ Result: {"results": [{"text": "(None,\n                      abilities  against_bug  against_dark               │
│ against_dragon  \\\n 0  ['Overgrow', 'Chlorophyll']          1.0           1.0             1.0   \n 1           │
│ ['Over...                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: The dataset contains information about 801 Pokémon. It includes various attributes for each Pokémon   
         such as:                                                                                                  
                                                                                                                   
         - Abilities                                                                                               
         - Effectiveness against different types (e.g., bug, dark, dragon, electric, fairy, etc.)                  
         - Base stats (attack, defense, special attack, special defense, speed, hp)                                
         - Classification, name, and Japanese name                                                                 
         - Physical attributes (height, weight)                                                                    
         - Other characteristics like generation, capture rate, base happiness, and whether the Pokémon is         
         legendary                                                                                                 
                                                                                                                   
         If you want, I can create some visualizations to better understand the data. What would you like to       
         explore first?

INFO     [agent] 🔢 tokens: 1773 total

In [7]:
query = "Can you aggregate the pokemons by type?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"# Aggregate the number of pokemons by type (for both type1 and type2)\n\n# Count            │
│ occurrences of type1\ncounts_type1 = df['type1'].value_counts()\n\n# Count occurrences of type2, ignoring       │
│ NaNs...                                                                                                         │
│ Result: {"results": [{"text": "water       131\nnormal      109\ngrass        98\nflying       98\npsychic      │
│ 82\nbug          77\npoison       66\nground       66\nfire         65\nrock         59\nfightin...             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: Here is the aggregated count of Pokémon by their types (considering both primary and secondary types):
                                                                                                                   
         - Water: 131                                                                                              
         - Normal: 109                                                                                             
         - Grass: 98                                                                                               
         - Flying: 98                                                                                              
         - Psychic: 82                                                                                             
         - Bug: 77                                                                                                 
         - Poison: 66                                                                                              
         - Ground: 66                                                                                              
         - Fire: 65                                                                                                
         - Rock: 59                                                                                                
         - Fighting: 53                                                                                            
         - Dark: 50                                                                                                
         - Electric: 48                                                                                            
         - Fairy: 47                                                                                               
         - Steel: 46                                                                                               
         - Dragon: 44                                                                                              
         - Ghost: 41                                                                                               
         - Ice: 38                                                                                                 
                                                                                                                   
         Would you like me to create a bar plot to visualize this distribution?

INFO     [agent] 🔢 tokens: 2211 total

### Gradio UI

You can use this provided Gradio UI to have a nicer user experience when chatting with your code generation agent. It will also give interesting information about the context stack, including the number of tokens used in the result, by the assistant, the user, and tools.

In [8]:
from lib.ui import ui

ui(coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://0be9c1f95bf767e3bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Your Turn

Now it's your turn to explore this unknown dataset. Ask questions and learn more about the information inside.

In [9]:
sbx = create_sandbox()
with open("unknown.csv", "rb") as f:
    content = f.read()
sbx.files.write("data.csv", content)

messages = []

ui(
    coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=ieu55eqyr43qqr4umb7b3)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://359754d3a3a0f30860.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
